In [29]:
# Это первая часть мини проекта по футбольной аналитики и визуализации
# Здесь представлен код для получения информации с ресурса https://understat.com , который позволяет довольно оперативно (в большинстве случаев) 
# получить информацию о прошедших футбольных матчах в различных первенствах
# thanks to https://github.com/mckayjohns/

In [30]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [31]:
# Каждый футбольный матч в конце URL имеет свой ID (пяти или шестизначный номер)
# Можно выбрать любой матч самостоятельно https://understat.com/
# или выбрать ID из примеров:

# Пример 1. Один из первых матчей 2022 года. Матч лидеров АПЛ Челси - Ливерпуль. https://understat.com/match/16578 (ID = 16578)
# Пример 2. Декабрьский матч московского Спартака в РПЛ. https://understat.com/match/16265 (ID = 16265)
# Пример 3. Матч Серии-А Милан-Рома. Forza Milan!  https://understat.com/match/16862 (ID = 16862)

# или найти любой интересующий Вас матч самостоятельно

# вводим этот ID ниже
base_url = 'https://understat.com/match/'
match = str(input('Please enter the match id: '))
url = base_url+match

Please enter the match id: 16578


In [32]:
# Используем BeautifulSoup чтобы запарсить страницу
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')
scripts = soup.find_all('script')

In [33]:
# находим нужные данные, в данном случае используем информацию об ударах по воротам
strings = scripts[1].string

In [34]:
# получаем и конвертируем JSON данные 
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')

data = json.loads(json_data)

In [35]:
# извлекаем нужные для анализа данные  
# h - обозначение домашней команды (home) 
# a - гостевая (away)

x = []
y = []
xG = []
result = []
team = []
h_a = []
minute = []
player = []
shotType = []
situation = []
data_away = data['a']
data_home = data['h']

for index in range(len(data_home)):
    for key in data_home[index]:
        if key == 'X':
            x.append(data_home[index][key])
        if key == 'Y':
            y.append(data_home[index][key])
        if key == 'h_team':
            team.append(data_home[index][key])
        if key == 'xG':
            xG.append(data_home[index][key])
        if key == 'result':
            result.append(data_home[index][key])
        if key == 'player':
            player.append(data_home[index][key])
        if key == 'minute':
            minute.append(data_home[index][key])            
        if key == 'h_a':
            h_a.append(data_home[index][key])
        if key == 'shotType':
            shotType.append(data_home[index][key])
        if key == 'situation':
            situation.append(data_home[index][key])            

for index in range(len(data_away)):
    for key in data_away[index]:
        if key == 'X':
            x.append(data_away[index][key])
        if key == 'Y':
            y.append(data_away[index][key])
        if key == 'a_team':
            team.append(data_away[index][key])
        if key == 'xG':
            xG.append(data_away[index][key])
        if key == 'result':
            result.append(data_away[index][key])
        if key == 'player':
            player.append(data_away[index][key])  
        if key == 'minute':
            minute.append(data_away[index][key])            
        if key == 'h_a':
            h_a.append(data_away[index][key])
        if key == 'shotType':
            shotType.append(data_away[index][key])
        if key == 'situation':
            situation.append(data_away[index][key])                                    

In [36]:
# немного преобразований
col_names = ['x','y','xG','result','team','player', 'minute', 'h_a', 'shottype', 'situation']
df = pd.DataFrame([x,y,xG,result,team, player, minute, h_a, shotType, situation],index=col_names)
df = df.T

In [37]:
# cразу изменим тип некоторых данных
df[['x', 'y', 'xG']] = df[['x', 'y', 'xG']].astype("float64")
df['minute'] = df['minute'].astype("int")

In [38]:
# взглянем на получившиеся данные 
df.sort_values(by = 'minute').head()

,x,y,xG,result,team,player,minute,h_a,shottype,situation
15,0.862,0.482,0.321807,SavedShot,Liverpool,Mohamed Salah,5,a,RightFoot,OpenPlay
16,0.892,0.498,0.676551,Goal,Liverpool,Sadio Mané,8,a,LeftFoot,OpenPlay
0,0.919,0.310,0.058264,BlockedShot,Chelsea,Mason Mount,15,h,RightFoot,OpenPlay
1,0.934,0.663,0.040274,MissedShots,Chelsea,Marcos Alonso,18,h,LeftFoot,OpenPlay
17,0.849,0.296,0.049754,BlockedShot,Liverpool,Trent Alexander-Arnold,23,a,RightFoot,OpenPlay


In [39]:
# контролируем наличие данных двух команд в датасете
df['team'].value_counts()

Chelsea      15
Liverpool    10
Name: team, dtype: int64

In [40]:
# подключаемся и сохраняем данные на Google Drive
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [41]:
df.to_csv('data.csv', index = False)
!cp data.csv "drive/My Drive/Colab Notebooks/Tableau"

In [42]:
# теперь можем переходить ко второй части - визуализациям в Tableau
# https://github.com/vladenec/projects_2021/tree/main/pet_MatchAnalytics